In [1]:
import datetime
import numpy as np
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from IPython.display import display

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
import logging
logging.basicConfig(level = logging.INFO, format = '[%(asctime)s][%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

In [50]:
def read_file(file_list):
    return [pd.read_csv(file_name+'.csv', iterator=True) for file_name in file_list]

def pd_engine(db_name='db_consumer_panel'):
    user = input('Username: ')
    password = input('Password: ')
    return create_engine(
        'mysql+mysqlconnector://%s:%s@127.0.0.1:3306/%s' % (user, password, db_name), 
        connect_args={'auth_plugin': 'mysql_native_password'}
    )

def load_file(engine, file_list, table_list, chunk_size=30000):
    dta_list = read_file(file_list)
    connection = engine.connect()
    for file, table, dta in zip(file_list, table_list, dta_list):
        count = 0
        logger.info('Loading from %s to %s...' % (file, table))
        connection.execute('DELETE FROM %s'%table)
        while True:
            try:
                dta_chunk = dta.get_chunk(chunk_size).drop(columns='Unnamed: 0')
                print(' - Loading %s-%s' % (count+1, count+chunk_size), end='')
                dta_chunk.to_sql(table, engine, index=False, if_exists='append')
                count += chunk_size
                print('\r', end='', flush=True)
            except StopIteration:
                logger.info(' # %s completed, with %s lines.' % (file, count-chunk_size+len(dta_chunk)))
                break
    return dta_chunk

In [6]:
engine = pd_engine()

Username: Jimmy
Password: ^Y&U*I9o0p-=


In [ ]:
table_list = ['raw_data_households', 'raw_data_products', 'raw_data_purchases', 'raw_data_trips']
file_list = ['dta_at_hh', 'dta_at_prod_id', 'dta_at_TC_upc', 'dta_at_TC']

debug = load_file(engine, file_list, table_list, chunk_size=40000)